# A1:

In [1]:
import cv2
import serial

ser = serial.Serial("COM10", 9600)

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

capture = cv2.VideoCapture(0)


# ---------------------

def read_pot_val():
    if ser.in_waiting > 0:
        value = ser.readline().decode("utf-8").strip()
        return int(value)
    
    return None
    
# ---------------------


prev_pot_value = 0

while True:
    isTrue, frame = capture.read()

    if not isTrue:
        break

    gray_img = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(gray_img, scaleFactor=1.1, minNeighbors=5,
                                          minSize=(30,30), flags=cv2.CASCADE_SCALE_IMAGE)
    
    x_s = 0
    y_s = 0
    w_s = 0
    h_s = 0

    area = 0
    
    for (x,y,w,h) in faces: 
        if (w * h) > area:
            x_s = x
            y_s = y
            w_s = w
            h_s = h

            area = w * h

    pot_value = read_pot_val()

    if pot_value is not None:
        prev_pot_value = pot_value
    else:
        pot_value = prev_pot_value


    line = int((pot_value/1023) * frame.shape[1])

    cv2.line(frame, (line, 0), (line, frame.shape[0]), (255,0,0), 2)

    
    if area > 0 and area > 500:

        if x_s <= line <= x_s + w_s:
            cv2.rectangle(frame, (x,y), (x+w, y+h), (0,255,255), 2)
        else:
            cv2.rectangle(frame, (x,y), (x+w, y+h), (0,0,255), 2)

    cv2.imshow("Video", frame)
    k = cv2.waitKey(5)

    if k == ord('q'):
        break

ser.close()
capture.release()
cv2.destroyAllWindows()

A2:

In [ ]:
import cv2
import serial
import time
ser = serial.Serial("COM10", 9600)

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

capture = cv2.VideoCapture(0)


# ---------------------

def read_pot_val():
    if ser.in_waiting > 0:
        value = ser.readline().decode("utf-8").strip()
        return int(value)
    
    return None
    
# ---------------------

def control_servo(position):
    ser.write(f"{position}".encode())

# ---------------------

servo_open = False
last_face_detected_time = time.time()

prev_pot_value = 0

while True:
    isTrue, frame = capture.read()

    if not isTrue:
        break

    gray_img = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(gray_img, scaleFactor=1.1, minNeighbors=5,
                                          minSize=(30,30), flags=cv2.CASCADE_SCALE_IMAGE)
    
    x_s = 0
    y_s = 0
    w_s = 0
    h_s = 0

    area = 0
    face_detected = False
    
    for (x,y,w,h) in faces: 
        if (w * h) > area:
            x_s = x
            y_s = y
            w_s = w
            h_s = h

            area = w * h
            face_detected = True

    #  pot_value = read_pot_val()

    # if pot_value is not None:
    #     prev_pot_value = pot_value
    # else:
    #     pot_value = prev_pot_value


    # line = int((pot_value/1023) * frame.shape[1])

    line = frame.shape[1] // 2

    cv2.line(frame, (line, 0), (line, frame.shape[0]), (255,0,0), 2)


    if face_detected and x_s <= line <= x_s + w_s:
        cv2.rectangle(frame, (x,y), (x+w, y+h), (0,255,255), 2)
    
        if not servo_open:
            ser.write(b'180')
            servo_open = True

        last_face_detected_time = time.time()
    
    else:
        cv2.rectangle(frame, (x,y), (x+w, y+h), (0,0,255), 2)

        if servo_open and (time.time() - last_face_detected_time > 2):
            ser.write(b'0')
            servo_open = False
    
    # if area > 0 and area > 500:

    #     if x_s <= line <= x_s + w_s:
    #         cv2.rectangle(frame, (x,y), (x+w, y+h), (0,255,255), 2)
    #     else:
    #         cv2.rectangle(frame, (x,y), (x+w, y+h), (0,0,255), 2)

    cv2.imshow("Video", frame)
    k = cv2.waitKey(5)

    if k == ord('q'):
        break

ser.close()
capture.release()
cv2.destroyAllWindows()

A3:

In [ ]:
import cv2
import serial
import time
ser = serial.Serial("COM10", 9600)

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

capture = cv2.VideoCapture(0)

# ---------------------

def control_servo(position):
    ser.write(f"{position}\n".encode())

# ---------------------

while True:
    isTrue, frame = capture.read()

    if not isTrue:
        break

    gray_img = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(gray_img, scaleFactor=1.1, minNeighbors=5,
                                          minSize=(30,30), flags=cv2.CASCADE_SCALE_IMAGE)
    
    x_s = 0
    y_s = 0
    w_s = 0
    h_s = 0

    area = 0
    face_detected = False
    
    for (x,y,w,h) in faces: 
        if (w * h) > area:
            x_s = x
            y_s = y
            w_s = w
            h_s = h

            area = w * h

    if area > 0:
        cv2.rectangle(frame, (x,y), (x+w, y+h), (0,0,255), 2)

        face_center_x = x_s + w_s // 2
        cv2.line(frame, (face_center_x, 0), (face_center_x, frame.shape[0]), (255,0,0), 2)

        servo_position = int((face_center_x / frame.shape[1]) * 180)

        control_servo(servo_position)
    
    cv2.imshow("Video", frame)
    k = cv2.waitKey(5)

    if k == ord('q'):
        break

ser.close()
capture.release()
cv2.destroyAllWindows()